In [2]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import math
from sklearn.metrics import confusion_matrix 
from scipy import stats
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

import statsmodels.api as sm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
# read data
claim_df =  pd.read_csv("./data/ClaimLevel.csv")
policy_df = pd.read_csv("./data/PolicyLevel.csv")

"""
                        # Data Preparation & Feature generation#
Generate a single record per policy that retains max information possible
"""
claim_df['CustomerPaidAmount'] = claim_df['ClaimedAmount'] - claim_df['PaidAmount']
claim_df_grouped = claim_df.groupby(['PolicyId'])
claim_per_policy_df = claim_df_grouped[['ClaimedAmount', 'PaidAmount', 'CustomerPaidAmount']].sum()
claim_per_policy_df['ClaimsCount'] = claim_df_grouped.size()
claim_per_policy_df['AvgClaimAmount'] = claim_df_grouped[['ClaimedAmount']].mean()
claim_per_policy_df['AvgPaidAmount'] = claim_df_grouped[['PaidAmount']].mean()
claim_per_policy_df['AvgCustomerPaidAmount'] = claim_df_grouped[['CustomerPaidAmount']].mean()
claim_per_policy_df['stdClaimAmount'] = claim_df_grouped[['ClaimedAmount']].std()
claim_per_policy_df['stdPaidAmount'] = claim_df_grouped[['PaidAmount']].std()
claim_per_policy_df['stdCustomerPaidAmount'] = claim_df_grouped[['CustomerPaidAmount']].std()
claim_per_policy_df['VarianceClaimAmount'] = claim_df_grouped[['ClaimedAmount']].var()
claim_per_policy_df['VariancePaidAmount'] = claim_df_grouped[['PaidAmount']].var()
claim_per_policy_df['VarianceCustomerPaidAmount'] = claim_df_grouped[['CustomerPaidAmount']].var()
claim_per_policy_df['maxClaimAmount'] = claim_df_grouped[['ClaimedAmount']].max()
claim_per_policy_df['minClaimAmount'] = claim_df_grouped[['ClaimedAmount']].min()
claim_per_policy_df['medianClaimAmount'] = claim_df_grouped[['ClaimedAmount']].median()
claim_per_policy_df['maxCustomerPaidAmount'] = claim_df_grouped[['ClaimedAmount']].max()
claim_per_policy_df['minCustomerPaidAmount'] = claim_df_grouped[['ClaimedAmount']].min()
claim_per_policy_df['medianCustomerPaidAmount'] = claim_df_grouped[['ClaimedAmount']].median()

policy_df = policy_df.join(claim_per_policy_df, on=['PolicyId'])

# Last seen Date: CancelDate if not null else '2016-12-31' (Last date in the data set)
policy_df['LastSeenDate'] = policy_df['CancelDate']
policy_df['LastSeenDate'].fillna(pd.to_datetime('2016-12-31'), inplace=True)

# Date conversions
policy_df['EnrollDate'] = pd.to_datetime(policy_df['EnrollDate'])
policy_df['CancelDate'] = pd.to_datetime(policy_df['CancelDate'])
policy_df['LastSeenDate'] = pd.to_datetime(policy_df['LastSeenDate'])

# Length of Stay "LOS"
policy_df['LOS'] = ((policy_df['LastSeenDate'] - policy_df['EnrollDate'])/np.timedelta64(1, 'M'))
policy_df = policy_df.round(decimals=2)

# Insurance Status: Active means 0 else 1
policy_df['InsuranceStatus'] = policy_df['CancelDate'].apply(lambda x: 1 if pd.notnull(x) else 0)

# drop null values
policy_df.dropna(axis='index', subset=['MonthlyPremium'], inplace=True)

# If no claims were made then all amounts are equal to 0 instead of NULL
for column in policy_df.columns :
    policy_df[column].fillna(0, inplace=True)

# display(claim_df.head())
display(policy_df.head(6))

,PolicyId,EnrollDate,CancelDate,MonthlyPremium,ClaimedAmount,PaidAmount,CustomerPaidAmount,ClaimsCount,AvgClaimAmount,AvgPaidAmount,...,VarianceCustomerPaidAmount,maxClaimAmount,minClaimAmount,medianClaimAmount,maxCustomerPaidAmount,minCustomerPaidAmount,medianCustomerPaidAmount,LastSeenDate,LOS,InsuranceStatus
0,92597,2010-12-07,1970-01-01,34.54,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2016-12-31,72.81,0
1,92808,2010-11-09,1970-01-01,18.54,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2016-12-31,73.73,0
2,93090,2010-11-11,1970-01-01,23.53,762.49,329.46,433.03,3.0,254.16,109.82,...,7971.91,510.48,100.00,152.01,510.48,100.00,152.01,2016-12-31,73.66,0
3,93122,2010-11-11,1970-01-01,44.02,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2016-12-31,73.66,0
4,93133,2010-11-11,1970-01-01,32.46,0.00,0.00,0.00,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2016-12-31,73.66,0
5,93258,2010-11-11,2016-05-24,21.66,33.99,30.60,3.39,3.0,11.33,10.20,...,0.00,11.33,11.33,11.33,11.33,11.33,11.33,2016-05-24,66.40,1


In [4]:
"""
                   # Data Description #
Churn --> 'C' --> coded as '1'
Non-Churn --> 'NC' --> coded as '0'
"""
total_policies = policy_df.shape[0]
curr_NC = policy_df[policy_df['InsuranceStatus']==0].shape[0]/float(total_policies)
curr_C = policy_df[policy_df['InsuranceStatus']==1].shape[0]/float(total_policies)
print("Total polices", total_policies)
print("NC:C = ",curr_NC, ":", curr_C)

Total polices 99995
NC:C =  0.8678433921696085 : 0.13215660783039151


In [5]:
# Sampling Data
churn_indices = policy_df[policy_df['InsuranceStatus']==1].index.values
nonchurn_indices = policy_df[policy_df['InsuranceStatus']==0].index.values
print(len(churn_indices), len(nonchurn_indices))

def sample_train_data(num_samples=None, churn_percent=None):
    """
    :param num_samples: total number of samples to return
    :param nc: ratio of non-churn in the return data set
    :param c: ration of churn in the return data set
    """
    # check to see if nc + c = 1
    churn_sample_space = churn_indices
    non_churn_sample_space = nonchurn_indices
        
    if num_samples is None and churn_percent is None:
        # use max samples possible (only 80% for train)
        churn_percent = curr_C 
        num_samples = int(len(churn_indices)*0.8/churn_percent)
    elif num_samples is None:
        num_samples = int(len(churn_indices)*0.8/churn_percent)
    
    if churn_percent==None:
        # default rate
        churn_percent = curr_C 
    elif churn_percent > 1:
        raise ValueError('Churn percent cannot be > 1')
    
    non_churn_percent = 1 - churn_percent
    nc_samples_count = int(num_samples * non_churn_percent)
    c_samples_count = int(num_samples * churn_percent)
    
    
    # prune train indices if looking for test set
    churn_sample_indices = np.random.choice(churn_sample_space, size=c_samples_count, replace=False)
    nonchurn_sample_indices = np.random.choice(non_churn_sample_space, size=nc_samples_count, replace=False)
    sample_indices = np.concatenate((churn_sample_indices, nonchurn_sample_indices))
    # shuffle
    np.random.shuffle(sample_indices)
    sample = policy_df.loc[sample_indices, :]
    return sample

def sample_test_data(skip_indices):
    
    churn_sample_space = np.setdiff1d(churn_indices, skip_indices)
    nonchurn_sample_space = np.setdiff1d(nonchurn_indices, skip_indices)
    churn_sample_indices = churn_sample_space
    nonchurn_sample_indices = np.random.choice(nonchurn_sample_space, size=len(churn_sample_indices), replace=False)
    sample_indices = np.concatenate((churn_sample_indices, nonchurn_sample_indices))
    # shuffle
    np.random.shuffle(sample_indices)
    sample = policy_df.loc[sample_indices, :]
    return sample
    
# test case
test = True
if test == False:
#     sample = sample_data(100, 0.8, 0.2)
    sample = sample_data()
    sample.head()
    sample_total_policies = sample.shape[0]
    NC = sample[sample['InsuranceStatus']==0].shape[0]/float(sample_total_policies)
    C = sample[sample['InsuranceStatus']==1].shape[0]/float(sample_total_policies)
    print("Total polices", total_policies)
    print("NC:C = ",NC, ":", C)

13215 86780


In [6]:
# data set 1: contains NC and C in the same ratio as the given in the original data and num_samples = 0.8%
train_set1_df = sample_train_data()
test_set1_df = sample_test_data(skip_indices=train_set1_df.index.values)

# # data set 2: NC:C = 1:1 and contains max samples possible
train_set2_df = sample_train_data(churn_percent=0.5)
test_set2_df = sample_test_data(skip_indices=train_set2_df.index.values)


# # data set 3: NC:C = 1:2 and contains max samples possible
train_set3_df = sample_train_data(churn_percent=0.66)
test_set3_df = sample_test_data(skip_indices=train_set3_df.index.values)

# # data set 4: NC:C = 2:1 and contains max samples possible
train_set4_df = sample_train_data(churn_percent=0.33)
test_set4_df = sample_test_data(skip_indices=train_set4_df.index.values)

# # data set 5: NC:C = 4:1 and contains max samples possible
train_set5_df = sample_train_data(churn_percent=0.20)
test_set5_df = sample_test_data(skip_indices=train_set5_df.index.values)

In [7]:
select_columns = list(
    set(policy_df.columns) - set(['PolicyId', 'EnrollDate', 'CancelDate', 'InsuranceStatus', 'LastSeenDate',
#                                  'ClaimedAmount', 'PaidAmount', 'CustomerPaidAmount' , 
#                                  'AvgClaimAmount', 'AvgPaidAmount', 'stdClaimAmount', 'stdPaidAmount'
#                                   'minClaimAmount', 'maxClaimAmount', 'medianClaimAmount'
                                 ]))

datasets  = [
        [train_set1_df, test_set1_df],
        [train_set2_df, test_set2_df],
        [train_set3_df, test_set3_df],
        [train_set4_df, test_set4_df],
        [train_set5_df, test_set5_df]
        ]

In [8]:
"""
                                        # RF Model #
"""
from sklearn.ensemble import RandomForestClassifier as RF

i = 1
for dataset in datasets:
    print("train Data set :", i)
    i += 1
    X_train = dataset[0][select_columns]
    X_test = dataset[1][select_columns]
    y_train = dataset[0][['InsuranceStatus']]
    y_test = dataset[1][['InsuranceStatus']]
    X_train, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_test)
    for weight in [0.5, 0.75, 1, 1.5, 2, 3, 4]:
        model = RF(n_estimators=100, class_weight={0:1,1:weight})
        model.fit(X_train, y_train)
        y_predicted = model.predict(X_test)
        print("\t Weight ", weight)
        print("\t Test accuracy",model.score(X_test, y_test))
        print("\t tn, fp, fn, tp ",confusion_matrix(y_test, y_predicted).ravel())

train Data set : 1
	 Weight  0.5
	 Test accuracy 0.50056753689
	 tn, fp, fn, tp  [2642    1 2639    4]
	 Weight  0.75
	 Test accuracy 0.500756715853
	 tn, fp, fn, tp  [2643    0 2639    4]
	 Weight  1
	 Test accuracy 0.5
	 tn, fp, fn, tp  [2641    2 2641    2]
	 Weight  1.5
	 Test accuracy 0.50056753689
	 tn, fp, fn, tp  [2643    0 2640    3]
	 Weight  2
	 Test accuracy 0.500756715853
	 tn, fp, fn, tp  [2642    1 2638    5]
	 Weight  3
	 Test accuracy 0.500189178963
	 tn, fp, fn, tp  [2642    1 2641    2]
	 Weight  4
	 Test accuracy 0.500378357927
	 tn, fp, fn, tp  [2643    0 2641    2]
train Data set : 2
	 Weight  0.5
	 Test accuracy 0.583238743852
	 tn, fp, fn, tp  [1191 1452  751 1892]
	 Weight  0.75
	 Test accuracy 0.598183881952
	 tn, fp, fn, tp  [1507 1136  988 1655]
	 Weight  1
	 Test accuracy 0.576428301173
	 tn, fp, fn, tp  [1317 1326  913 1730]
	 Weight  1.5
	 Test accuracy 0.580022701476
	 tn, fp, fn, tp  [2123  520 1700  943]
	 Weight  2
	 Test accuracy 0.566780174045
	 tn,

In [9]:
"""
                                        # Logistic Model #
"""
from sklearn.linear_model import LogisticRegression

i = 1
for dataset in datasets:
    print("train Data set :", i)
    i += 1
    X_train = dataset[0][select_columns]
    X_test = dataset[1][select_columns]
    y_train = dataset[0][['InsuranceStatus']]
    y_test = dataset[1][['InsuranceStatus']]
    X_train, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_test)
    for weight in [0.5, 0.75, 1, 1.5, 2, 3, 4]:
        point_weights = dataset[0].InsuranceStatus.apply(lambda x: 1 if x == 0 else weight)
        model = LogisticRegression(penalty='l1',)
        model.fit(X_train, y_train, point_weights)
        y_predicted = model.predict(X_test)
        print("\t Weight on disenrolled polcies", weight)
        print("\t Test accuracy",model.score(X_test, y_test))
        print("\t tn, fp, fn, tp ",confusion_matrix(y_test, y_predicted).ravel())

train Data set : 1
	 Weight on disenrolled polcies 0.5
	 Test accuracy 0.5
	 tn, fp, fn, tp  [2643    0 2643    0]
	 Weight on disenrolled polcies 0.75
	 Test accuracy 0.50056753689
	 tn, fp, fn, tp  [2643    0 2640    3]
	 Weight on disenrolled polcies 1
	 Test accuracy 0.500756715853
	 tn, fp, fn, tp  [2642    1 2638    5]
	 Weight on disenrolled polcies 1.5
	 Test accuracy 0.501324252743
	 tn, fp, fn, tp  [2642    1 2635    8]
	 Weight on disenrolled polcies 2
	 Test accuracy 0.500945894816
	 tn, fp, fn, tp  [2640    3 2635    8]
	 Weight on disenrolled polcies 3
	 Test accuracy 0.50170261067
	 tn, fp, fn, tp  [2637    6 2628   15]
	 Weight on disenrolled polcies 4
	 Test accuracy 0.503216042376
	 tn, fp, fn, tp  [2632   11 2615   28]
train Data set : 2
	 Weight on disenrolled polcies 0.5
	 Test accuracy 0.50170261067
	 tn, fp, fn, tp  [2640    3 2631   12]
	 Weight on disenrolled polcies 0.75
	 Test accuracy 0.605372682558
	 tn, fp, fn, tp  [2010  633 1453 1190]
	 Weight on disenro

In [10]:
"""
                                        # neural nets Model #
"""
from sklearn.neural_network import MLPClassifier

datasets  = [
        [train_set1_df, test_set1_df],
        [train_set2_df, test_set2_df],
        [train_set3_df, test_set3_df],
        [train_set4_df, test_set4_df]
        ]

i = 1
for dataset in datasets:
    print("train Data set :", i)
    i += 1
    X_train = dataset[0][select_columns]
    X_test = dataset[1][select_columns]
    y_train = dataset[0][['InsuranceStatus']]
    y_test = dataset[1][['InsuranceStatus']]
    X_train, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_test)
    model = MLPClassifier(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(16, 32, 24, 8), random_state=1, 
                         activation='relu')
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
    print("\t Train",model.score(X_train, y_train))
    print("\t tn, fp, fn, tp ",confusion_matrix(y_test, y_predicted).ravel())


train Data set : 1
	 Train 0.869343467173
	 tn, fp, fn, tp  [2640    3 2626   17]
train Data set : 2
	 Train 0.624385168369
	 tn, fp, fn, tp  [1670  973 1030 1613]
train Data set : 3
	 Train 0.692077168009
	 tn, fp, fn, tp  [ 504 2140  215 2429]
train Data set : 4
	 Train 0.693241766818
	 tn, fp, fn, tp  [2540  104 2354  290]


In [11]:
print("Length of train sets")
print(len(train_set1_df))
print(len(train_set2_df))
print(len(train_set3_df))
print(len(train_set4_df))
print(len(train_set5_df))
print("Length of test sets")
print(len(test_set1_df))
print(len(test_set2_df))
print(len(test_set3_df))
print(len(test_set4_df))
print(len(test_set5_df))

Length of train sets
79996
21144
16017
32035
52860
Length of test sets
5286
5286
5288
5288
5286


In [12]:
print(len(policy_df))

99995
